<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# Project 3 Web APIs and NLP

_Authors: Joel Quek (SG)_

# [Delete Later]

For project 3, your goal is two-fold:

1. Using Pushshift's API, you'll collect posts from two subreddits of your choosing.
2. You'll then use NLP to train a classifier on which subreddit a given post came from. This is a binary classification problem.

# Problem Statement

Do Reddit posts affect stock index movements?

Model the effect of reddit posts on the price movements of QQQ and SPY

# Exploratory Data Analysis

## Import Libraries

In [2]:
#All libraries used in this project are listed here
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords

import re
from bs4 import BeautifulSoup 

from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV,cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import roc_auc_score, make_scorer, recall_score, precision_score,accuracy_score

# Open Scraped Datasets

In [3]:
investing_df = pd.read_csv('datasets/investing.csv')
stockmarket_df = pd.read_csv('datasets/stockmarket.csv')
wallstreetbets_df = pd.read_csv('datasets/wallstreetbets.csv')

## r/investing

In [7]:
investing_df.shape

(5248, 72)

In [4]:
investing_df[['subreddit', 'author', 'selftext', 'title']].head()

,subreddit,author,selftext,title
0,investing,hydrocyanide,https://www.treasurydirect.gov/savings-bonds/i...,New Series I savings bond rate is 6.89% (0.40%...
1,investing,KyloRenSucks,[removed],Treasury announces new Series I bond rate of 6...
2,investing,Electronic_Ad_701,[removed],Investment question? New to this and 19 I’m eager
3,investing,Quiet-Literature2251,"Hey, this is my current plan. I'm in sales and...",Making substantial money now but not sure what...
4,investing,4619,[removed],What happens in an emerging markets ETF when a...


## r/stockmarket

In [8]:
stockmarket_df.shape

(5245, 81)

In [5]:
stockmarket_df[['subreddit', 'author', 'selftext', 'title']].head()

,subreddit,author,selftext,title
0,StockMarket,nikola28,NaN,Avatrade Leverage Trading: Everything You Need...
1,StockMarket,Philip19967,NaN,Fobi Signs Strategic Partnership Agreement Wit...
2,StockMarket,ad3lyt,NaN,The Metaverse Is Dying
3,StockMarket,irahuo,NaN,Dogecoin increases by 29% as the Musk-Fueled M...
4,StockMarket,predictany007,\n\n* **CNBC’s Jim Cramer on Monday warned in...,‘Hold your nose and sell’ to brace for a possi...


## r/wallstreetbets

In [9]:
wallstreetbets_df.shape

(1249, 83)

In [6]:
wallstreetbets_df[['subreddit', 'author', 'selftext', 'title']].head()

,subreddit,author,selftext,title
0,wallstreetbets,Ellarael,This is what buffet was talking about right? N...,bullish on meta
1,wallstreetbets,Remote-Ad-3,NaN,Avis budget group
2,wallstreetbets,Lija_2016,NaN,Lillian Finance [$LY] is getting listed on TWO...
3,wallstreetbets,infurnus89,NaN,$CINE Cineworld settles with landlords. Price ...
4,wallstreetbets,0x11C3P,NaN,JOLTS + Others


# Index Data [QQQ and SPY]